In [1]:
import pandas as pd
import sys
from tqdm import tqdm
import time
from multiprocessing import Pool, cpu_count
import numpy as np
import os
from openai import OpenAI

# Agregar path de funciones
sys.path.append('../functions')
from openai_law_matcher import (
    filter_official_regex_matches_parallel,
    apply_openai_law_matching_deduplicated
)

In [2]:
# Verificar que la API key está configurada
api_key = os.getenv('OPENAI_API_KEY')

# Inicializar cliente de OpenAI
client = OpenAI(api_key=api_key)

In [3]:
df = pd.read_csv('../../data/03_extracted/legal_docs_generic.csv')
df = df[~df['art_id'].str.contains('_TRANS', na=False)]

In [4]:
df.head()

,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
0,6D0C4493,6D0C4493_2,Ley de Centros Penitenciarios de la Ciudad de ...,LAW_MENTION,LEGAL_DOCS,de Sanciones Administrativas y de Integración ...,30,1
1,6D0C4493,6D0C4493_3,Ley de Centros Penitenciarios de la Ciudad de ...,LAW_MENTION,LEGAL_DOCS,"Para los efectos del presente Reglamento , ade...",14,30
2,6D0C4493,6D0C4493_3,Ley de Centros Penitenciarios,LAW_MENTION,LEGAL_DOCS,Sanciones Administrativas de la Ciudad de Méxi...,30,30
3,6D0C4493,6D0C4493_3,Ley de Centros Penitenciarios de la Ciudad de ...,LAW_MENTION,LEGAL_DOCS,de México ; Constitución Federal : Constitució...,30,30
4,6D0C4493,6D0C4493_3,Ley de Centros Penitenciarios de la Ciudad de ...,LAW_MENTION,LEGAL_DOCS,otras expresiones no visualizadas en el acróni...,30,20


In [5]:
law_mentions = df[df['entity_label'] == 'LAW_MENTION']
law_mentions['entity_text'].unique()

array(['Ley de Centros Penitenciarios de la Ciudad de México y demás normativa aplicable',
       'Ley de Centros Penitenciarios de la Ciudad de México',
       'Ley de Centros Penitenciarios', ...,
       'Ley de Aguas Nacionales y demás disposiciones jurídicas aplicables',
       'Ley de Procedimiento Administrativo del Distrito Federal y sus Reglamentos',
       'Ley para Prevenir y Erradicar la Discriminación el Distrito Federal'],
      dtype=object)

In [6]:
# Lista con solo los códigos de la Ciudad de México
leyes_completas = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')
ley = leyes_completas[leyes_completas['nombre_normalized'].str.startswith('LEY', na=False)]
ley



,nombre,fecha_publicacion,fecha_actualizacion,link_pdf,link_docx,source_url,gov_level,nombre_normalized,doc_id
1,Ley de Proteccion de Datos Personales en Poses...,03/10/08,NaN,https://data.consejeria.cdmx.gob.mx//images/le...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYDEPROTECCIONDEDATOSPERSONALESENPOSESIONDESU...,80360428
2,Ley de Acceso de las Mujeres a una Vida Libre ...,25/06/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYDEACCESODELASMUJERESAUNAVIDALIBREDEVIOLENCI...,74FF23BF
3,Ley de los Derechos de Ninas Ninos y Adolescen...,02/04/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYDELOSDERECHOSDENINASNINOSYADOLESCENTESDELAC...,FB6CCCAE
4,Ley para el Reconocimiento y la Atencion de la...,31/03/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYPARAELRECONOCIMIENTOYLAATENCIONDELASPERSONA...,6C800A8E
5,Ley para la Celebracion de Espectaculos Public...,25/03/25,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,LEYPARALACELEBRACIONDEESPECTACULOSPUBLICOSDELA...,298E1252
...,...,...,...,...,...,...,...,...,...
641,"Ley sobre el Escudo, la Bandera y el Himno Nac...",08/02/84,01/04/24,NaN,NaN,NaN,FEDERAL,LEYSOBREELESCUDOLABANDERAYELHIMNONACIONALES,9DDF0C1E
642,Ley sobre Elaboracion y Venta de Cafe Tostado,25/05/72,10/12/04,NaN,NaN,NaN,FEDERAL,LEYSOBREELABORACIONYVENTADECAFETOSTADO,761D1F2F
643,Ley sobre la Aprobacion de Tratados Internacio...,02/09/04,NaN,NaN,NaN,NaN,FEDERAL,LEYSOBRELAAPROBACIONDETRATADOSINTERNACIONALESE...,90CCD942
644,Ley sobre la Celebracion de Tratados,02/01/92,20/05/21,NaN,NaN,NaN,FEDERAL,LEYSOBRELACELEBRACIONDETRATADOS,703EDD4C


## Filtrar menciones que ya tienen match con regex oficiales

Antes de procesar con LLM (que es costoso), filtramos las menciones que ya pueden ser identificadas con los patrones de regex oficiales de CDMX y federales.


In [7]:
# Filtrar menciones que ya tienen match con regex
law_mentions_sin_match, law_mentions_con_match, stats = filter_official_regex_matches_parallel(
    df=law_mentions,
    entity_text_col='entity_text',
    n_jobs=None  # Usa todos los cores
)

# print(f"\n✓ Filtrado completado")
# print(f"  - Menciones CON match en regex: {stats['con_match_regex']}")
# print(f"  - Menciones SIN match (para LLM): {stats['sin_match_regex']}")



=== FILTRANDO MENCIONES CON REGEX OFICIALES (PARALELO) ===
Total de menciones a analizar: 4324
Total de patrones regex a aplicar: 784
Usando 8 procesos paralelos
Procesando 8 chunks en paralelo...


/opt/anaconda3/envs/cdmx_kg/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



=== RESULTADOS DEL FILTRADO ===
Total de menciones: 4324
Con match en regex (filtradas): 1720
Sin match en regex (para LLM): 2604
Porcentaje filtrado: 39.8%

=== EJEMPLOS DE MENCIONES FILTRADAS (YA TIENEN MATCH) ===
1. Ley de Centros Penitenciarios de la Ciudad de México y demás normativa aplicable
   Match: LEY DE CENTROS PENITENCIARIOS DE LA CIUDAD DE MÉXICO
2. Ley de Centros Penitenciarios de la Ciudad de México
   Match: LEY DE CENTROS PENITENCIARIOS DE LA CIUDAD DE MÉXICO
3. Ley de Centros Penitenciarios de la Ciudad de México
   Match: LEY DE CENTROS PENITENCIARIOS DE LA CIUDAD DE MÉXICO
4. Ley de Centros Penitenciarios de la Ciudad de México
   Match: LEY DE CENTROS PENITENCIARIOS DE LA CIUDAD DE MÉXICO
5. Ley de Archivos de la Ciudad de México y demás normativa aplicable y sólo podrá 
   Match: LEY DE ARCHIVOS DE LA CIUDAD DE MÉXICO

=== EJEMPLOS DE MENCIONES SIN MATCH (PARA LLM) ===
1. Ley de Centros Penitenciarios
2. Ley de Centros Penitenciarios
3. Ley de Centros Penitencia

In [8]:
stats

{'total': 4324,
 'con_match_regex': 1720,
 'sin_match_regex': 2604,
 'porcentaje_filtrado': 39.77798334875116}

## Matching con OpenAI para menciones sin match

Ahora procesamos con LLM solo las menciones que NO tuvieron match con los regex oficiales.


In [9]:
# Aplicar matching con OpenAI solo a las menciones SIN match en regex

# Matching deduplicado
results_df = apply_openai_law_matching_deduplicated(
    mentions_df=law_mentions_sin_match,
    cdmx_laws_df=ley,
    client=client,
    entity_text_col='entity_text',
    art_id_col='art_id',
    delay_seconds=1.5,  # Reducido de 1.0
    temperature=0.2
)

print(f"\nMatching con LLM completado. Total de resultados: {len(results_df)}")

=== MATCHING DE LEYES CON OPENAI (OPTIMIZADO - DEDUPLICADO) ===
Total de menciones (con duplicados): 2604
Filas válidas: 2604 (de 2604 total)

Menciones ÚNICAS a procesar: 1103
 Reducción: 1501 llamadas ahorradas
Ahorro de tiempo estimado: 37.5 minutos
Tiempo estimado total: 27.6 minutos
Leyes oficiales CDMX en catálogo: 470
Temperatura del modelo: 0.2

Procesando menciones únicas...
  Progreso: 10/1103 menciones únicas procesadas
  Tiempo transcurrido: 0.4 minutos
  Tiempo restante estimado: 27.3 minutos

  Progreso: 20/1103 menciones únicas procesadas
  Tiempo transcurrido: 0.8 minutos
  Tiempo restante estimado: 27.1 minutos

  Progreso: 30/1103 menciones únicas procesadas
  Tiempo transcurrido: 1.4 minutos
  Tiempo restante estimado: 26.8 minutos

  Progreso: 40/1103 menciones únicas procesadas
  Tiempo transcurrido: 2.0 minutos
  Tiempo restante estimado: 26.6 minutos

  Progreso: 50/1103 menciones únicas procesadas
  Tiempo transcurrido: 2.9 minutos
  Tiempo restante estimado: 26

In [10]:
results_df.to_csv('../../data/04_matched/law_mentions_matched.csv', index=False)